## Importación de librerías

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from scipy.io import arff
import random
from collections import OrderedDict
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

## Lectura de modelos guardados

In [ ]:
# cargamos los modelos
dt_model = joblib.load(os.path.join('modelos','Modelo_DT.pkl'))
lr_model = joblib.load(os.path.join('modelos','Modelo_LR.pkl'))
rf_model = joblib.load(os.path.join('modelos','Modelo_RF.pkl'))
xgb_model = joblib.load(os.path.join('modelos','Modelo_XGB.pkl'))
cv_svm_model = joblib.load(os.path.join('modelos','Modelo_SVM.pkl'))
cs_svm_ga_model = joblib.load(os.path.join('modelos','Modelo_GA_SVM.pkl'))

## Lectura de dataset imputado

In [ ]:
# se carga dataset imputado
median_imputed_df = pd.read_pickle(os.path.join('results','median_imputed_data.pkl'))

## Evaluación de modelos

In [ ]:
# Modelamiento de la data usando un diccionario de datasets y modelos
def perform_model_evaluation(_models_, imputed_df, verbose=False, k_folds=5):
    # 7 metricas, usando K-Folds, solo se evalua (no fit)
    # en model_results guardaremos los resultados por clasificador y datasets
    model_results = OrderedDict()

    # Iteramos sobre los clasificadores
    for model_name, clf in _models_.items():
        if verbose: print("-" * 120, "\n", "Model: " + '\033[1m' + model_name + '\033[0m' + " Classifier")
        imputer_results = OrderedDict()

        # hacemos la division del dataframe en variables y etiquetas
        features_df, labels_df = split_features_labels(imputed_df)

        df_index = 0
        if verbose: print('\t\tDataset: ' + '\033[1m' + str(df_index + 1) + 'year' + '\033[0m')
        # Ejecutamos la validación cruzada K-fold en los sets de entranamiento y test
        X_train_list, y_train_list, X_test_list, y_test_list = kfold_cv(k_folds, features_df,
                                                                        labels_df, verbose)

        metrics = OrderedDict()

        # incializamos las metricas a guardar
        accuracy_list = np.zeros([k_folds])
        precision_list = np.zeros([k_folds, 2])
        recall_list = np.zeros([k_folds, 2])
        true_negs = np.zeros([k_folds])
        false_pos = np.zeros([k_folds])
        false_negs = np.zeros([k_folds])
        true_pos = np.zeros([k_folds])

        # Iteramos sobre los k-folds para el cálculo de las métricas
        for k in range(k_folds):
            X_train = X_train_list[k]
            y_train = y_train_list[k]
            X_test = X_test_list[k]
            y_test = y_test_list[k]

            # y predicción en el set de test
            y_test_predicted = clf.predict(X_test)
            # presentamos la matriz de confusión
            print(confusion_matrix(y_test_predicted, y_test))

            # guardamos accuracy y recall
            _accuracy_ = accuracy_score(y_test, y_test_predicted, normalize=True)
            accuracy_list[k] = _accuracy_
            _recalls_ = recall_score(y_test, y_test_predicted, average=None)
            recall_list[k] = _recalls_

            # guardamos precision
            _precisions_ = precision_score(y_test, y_test_predicted, average=None)
            precision_list[k] = _precisions_

            # calculamos la matriz de confusión
            _confusion_matrix_ = confusion_matrix(y_test, y_test_predicted)
            mlp_cm = confusion_matrix(y_test, y_test_predicted)

            # guardamos demás valores: TN, FP, FN, TP
            true_negs[k] = _confusion_matrix_[0][0]
            false_pos[k] = _confusion_matrix_[0][1]
            false_negs[k] = _confusion_matrix_[1][0]
            true_pos[k] = _confusion_matrix_[1][1]

        # Hacemos la media en el caso de más datasets
        metrics['Accuracy'] = np.mean(accuracy_list)
        metrics['Precisions'] = np.mean(precision_list, axis=0)
        metrics['Recalls'] = np.mean(recall_list, axis=0)
        metrics['TN'] = np.mean(true_negs)
        metrics['FP'] = np.mean(false_pos)
        metrics['FN'] = np.mean(false_negs)
        metrics['TP'] = np.mean(true_pos)

        # presentamos algunos valores
        if verbose:
            print('\t\t\tAccuracy:', metrics['Accuracy'])
            print('\t\t\tPrecision:', metrics['Precisions'])
            print('\t\t\tRecall:', metrics['Recalls'])

        # guardamos en el diccionario
        model_results[model_name] = metrics

    # presentamos la matriz de confusión en modo mapa de calor
    sns.heatmap(mlp_cm, annot=True,
                xticklabels=['Non Bankrupt', 'Bankrupt'],
                yticklabels=['Non Bankrupt', 'Bankrupt'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    return model_results

In [ ]:
# Hacer ranking por cada métrica
def perform_model_ranking_acc(models, imputers, results):
    column_headers = ['-','Accuracy'] 
    rows = []
    for model_name, model_details in results.items():
        row = [model_name]
        row.append(model_details['Accuracy'])
        rows.append(row)
    results_df = pd.DataFrame(data=rows, columns = column_headers)
    return results_df

def perform_model_ranking_prec(models, imputers, results):
    column_headers = ['-','Precisions']
    rows = []
    for model_name, model_details in results.items():
        row = [model_name]
        row.append(model_details['Precisions'])
        rows.append(row)
    results_df = pd.DataFrame(data=rows, columns = column_headers)
    return results_df

def perform_model_ranking_rec(models, imputers, results):
    column_headers = ['-','Recalls']
    rows = []
    for model_name, model_details in results.items():
        row = [model_name]
        row.append(model_details['Recalls'])
        rows.append(row)
    results_df = pd.DataFrame(data=rows, columns = column_headers)
    return results_df

def perform_model_ranking_tn(models, imputers, results):
    column_headers = ['-','TN']
    rows = []
    for model_name, model_details in results.items():
        row = [model_name]
        row.append(model_details['TN'])
        rows.append(row)
    results_df = pd.DataFrame(data=rows, columns = column_headers)
    return results_df

def perform_model_ranking_fp(models, imputers, results):
    column_headers = ['-','FP']
    rows = []
    for model_name, model_details in results.items():
        row = [model_name]
        row.append(model_details['FP'])
        rows.append(row)
    results_df = pd.DataFrame(data=rows, columns = column_headers)
    return results_df

def perform_model_ranking_fn(models, imputers, results):
    column_headers = ['-','FN']
    rows = []
    for model_name, model_details in results.items():
        row = [model_name]
        row.append(model_details['FN'])
        rows.append(row)
    results_df = pd.DataFrame(data=rows, columns = column_headers)
    return results_df

def perform_model_ranking_tp(models, imputers, results):
    column_headers = ['-','FP']
    rows = []
    for model_name, model_details in results.items():
        row = [model_name]
        row.append(model_details['TP'])
        rows.append(row)
    results_df = pd.DataFrame(data=rows, columns = column_headers)
    return results_df

## Análisis comparativo

**Comparación**

**Tablas**

## Prducción de gráficos